<br>

# Técnicas Matemáticas para Big Data - Project 04
<br><br>

GROUP 06:
- Joana Daniela Carvalho Rodrigues - Nº 112926 - xx% Work Participation
- Liliana Paula Cruz Ribeiro - Nº 108713 - xx% Work Participation
- Willian Pegorin - Nº 122970 - xx% Work Participation

<br><br>
## **Q1:**  What is the numeric criteria that you may use to determine if a change in the algorithm produces improvements?

To determine if a change to the SOM algorithm leads to improvements, we must use quantitative performance measures. A commonly used metric is the **Quantization Error (QE)**, which measures the average distance between each input vector $x_i$ and its corresponding **Best Matching Unit (BMU)** after training. This metric evaluates how accurately the SOM represents the unput data distribution. The QE is given by the following expression:
$$
QE = \dfrac{1}{N}\sum_{i=1}^{N}\|x_i-w_{BMU(i)}\|
$$
where $N$ is the number of input samples and $w_{BMU(i)}$ is the weight vector of the BMU associated with $x_i$.

However, QE alone does not evaluate the preservation of the input space topology. For this reason, the Topographic Error (TE) is also commonly used. It measures the proportion of data samples for which the first and second BMUs are not adjacent in the map also quantifying topological distortions.

Formally, the TE is defined as:
$$
TE = \dfrac{1}{N} \sum_{i=1}^N u(x_i),
$$
where $u(x_i) = 1$ if the first and second BMUs of $x_i$ are not adjacent and $u(x_i) = 0$ otherwise. The TE takes values in the interval $[0,1]$, with lower values indicating better topological preservation.

An algorithmic change may be considered an improvement if it results in a lower QE while maintaining or reducing the TE.

<br><br>
## **Q2:** Write the version SOM1A, where you change the curve of the learning factor. Did you achieve improvements?

In version SOM1A, we changed the curve of the learning factor to a linear decay instead of exponencial in file "our_som1A", defined as:

$$
\alpha(t) = \alpha_0\left(1-\dfrac{t}{T}\right)
$$
where $\alpha_0$ is the initial learning rate and $T$ is the total number of training iterations.

In [ ]:
def decay_learning_rate(self, initial_learning_rate, iteration, num_iterations):
        return initial_learning_rate * (1 - iteration / num_iterations)

To evaluate the impact of this modification, we used two SOMs with the same initialization seed and number of epochs, but one was trained using the original exponencial decay and the other using the linear decay. To understand if there was an improvement, we used QE and TE.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import patches as patches
import matplotlib.lines as mlines

# reading data
data = pd.read_csv("cash-crops-nepal.csv")
agri_data = data.iloc[np.random.permutation(len(data))]
trunc_data = agri_data[["Area", "Production", "Yield"]]
trunc_data = trunc_data / trunc_data.max()

from our_som1A import SOM_A
from our_som1 import SOM

# som = SOM(x_size, y_size, num_features)
np.random.seed(42)
agri_som_lin = SOM_A(6,6,3)

# Initial weights
init_fig = plt.figure()
agri_som_lin.show_plot(init_fig, 1, 0)
plt.show()

agri_som_lin.train(trunc_data.values,
          num_epochs=200,
          init_learning_rate=0.01
          )

# Função para calcular métricas
def evaluate_som(som, data):
    """
    Calcula métricas de qualidade do SOM
    """
    # 1. Erro de Quantização (Quantization Error)
    quantization_error = 0
    for idx, row in data.iterrows():
        bmu, _ = som.find_bmu(row.values)
        quantization_error += np.sum((row.values - bmu) ** 2)
    quantization_error /= len(data)

    # 2. Erro Topográfico (Topographic Error)
    topographic_error = 0
    for idx, row in data.iterrows():
        # Encontrar os dois BMUs mais próximos
        distances = []
        for x in range(som.network_dimensions[0]):
            for y in range(som.network_dimensions[1]):
                w = som.net[x, y, :]
                dist = np.sum((row.values - w) ** 2)
                distances.append(((x, y), dist))

        # Ordenar por distância
        distances.sort(key=lambda x: x[1])
        bmu1, bmu2 = distances[0][0], distances[1][0]

        # Verificar se são adjacentes (Manhattan distance <= 1)
        if abs(bmu1[0] - bmu2[0]) + abs(bmu1[1] - bmu2[1]) > 1:
            topographic_error += 1

    topographic_error /= len(data)


    return {
        'quantization_error': quantization_error,
        'topographic_error': topographic_error
    }


metrics_lin = evaluate_som(agri_som_lin, trunc_data)

np.random.seed(42)
agri_som_exp = SOM(6, 6, 3)
agri_som_exp.train(trunc_data.values, num_epochs=200, init_learning_rate=0.01)
metrics_exp = evaluate_som(agri_som_exp, trunc_data)


print(f"\n{'Métrica':<30} {'Linear':<15} {'Exponencial':<15}")
print("-"*70)

print(f"{'QE':<30} {metrics_lin['quantization_error']:<15.6f} "
      f"{metrics_exp['quantization_error']:<15.6f}")

print(f"{'Topographic error':<30} {metrics_lin['topographic_error']:<15.6f} "
      f"{metrics_exp['topographic_error']:<15.6f}")

FileNotFoundError: [Errno 2] No such file or directory: 'cash-crops-nepal.csv'

These were the results:

| Metric | Linear | Exponencial |
| ------ | :----: | :---------: |
| QE | 0.004088 | 0.002417 |
| TE | 0.142857 | 0.152381 |

Observing the table above, the results indicate that the linear decay leads to a slight improvement in topographic preservation due to a lower TE. However, it has a higher QE.

Therefore, the modification doesn't result in an overall improvement. It seems that the linear decay reduces the learning rate to aggressively, resulting in a reduced ability to perform subtle weight adjustments toward the end of the training process. On the other hand, the exponential decay provides a better balance between global ordering and local convergence.

<br><br>
## **Q3:** Write the version SOM1B, where you change the curve of the deviation. Did you achieve improvements?

In the Self-Organizing Map algorithm, the neighborhood deviation σ(t) controls the spatial extent of the neighborhood function and regulates the balance between global ordering and local adaptation. Larger values of σ(t) promote smoother global updates, while smaller values emphasize local refinement around the Best Matching Unit (BMU). Therefore, the decay profile of σ(t) directly influences the convergence dynamics of the algorithm.

In the provided implementation, the deviation parameter σ(t) is implicitly represented by the neighborhood radius variable. The SOM1B version was implemented by modifying exclusively the decay function of this radius, replacing the original exponential decay with a linear decay combined with a lower bound, while keeping the Gaussian neighborhood function unchanged.


The following code snippet illustrates the modification introduced in the SOM1B version:
```python
<our_som1B.py>
class SOM_B:
...
    def decay_radius(self, iteration):
        #return self.init_radius * np.exp(-iteration / self.time_constant)
        # Decaimento linear
        radius = self.init_radius * (1 - iteration / self.time_constant)
        return max(radius, 1.0)

In [ ]:
### version SOM1B
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import patches as patches
import matplotlib.lines as mlines

# reading data
data = pd.read_csv("cash-crops-nepal.csv")
agri_data = data.iloc[np.random.permutation(len(data))]
trunc_data = agri_data[["Area", "Production", "Yield"]]
trunc_data = trunc_data / trunc_data.max()

from our_som1B import SOM_B
from our_som1 import SOM

# som = SOM(x_size, y_size, num_features)
np.random.seed(42)
agri_som_decay_radius_lin = SOM_B(6,6,3)

# Initial weights
init_fig = plt.figure()
agri_som_decay_radius_lin.show_plot(init_fig, 1, 0)
plt.show()

agri_som_decay_radius_lin.train(trunc_data.values,
          num_epochs=200,
          init_learning_rate=0.01
          )

# Function to compute SOM quality metrics
def evaluate_som(som, data):
    """
    Computes quality metrics for a trained Self-Organizing Map (SOM).

    Metrics:
    1. Quantization Error (QE)
    2. Topographic Error (TE)
    """

    # 1. Quantization Error (QE)
    quantization_error = 0.0
    for _, row in data.iterrows():
        bmu, _ = som.find_bmu(row.values)
        quantization_error += np.sum((row.values - bmu) ** 2)
    quantization_error /= len(data)

    # 2. Topographic Error (TE)
    topographic_error = 0.0
    for _, row in data.iterrows():
        # Find the two closest BMUs
        distances = []
        for x in range(som.network_dimensions[0]):
            for y in range(som.network_dimensions[1]):
                w = som.net[x, y, :]
                dist = np.sum((row.values - w) ** 2)
                distances.append(((x, y), dist))

        # Sort by distance
        distances.sort(key=lambda x: x[1])
        bmu1, bmu2 = distances[0][0], distances[1][0]

        # Check if the two BMUs are adjacent (Manhattan distance <= 1)
        if abs(bmu1[0] - bmu2[0]) + abs(bmu1[1] - bmu2[1]) > 1:
            topographic_error += 1

    topographic_error /= len(data)

    return {
        "quantization_error": quantization_error,
        "topographic_error": topographic_error
    }

metrics_radius_lin = evaluate_som(agri_som_decay_radius_lin, trunc_data)

np.random.seed(42)
agri_som_decay_radius_exp = SOM(6, 6, 3)
agri_som_decay_radius_exp.train(trunc_data.values, num_epochs=200, init_learning_rate=0.01)
metrics_radius_exp = evaluate_som(agri_som_decay_radius_exp, trunc_data)


print(f"\n{'Metric':<30} {'Radius Exponential':<15} {'Radius Linear':<15}")
print("-"*70)

print(f"{'QE':<30} {metrics_radius_exp['quantization_error']:<15.6f} "
      f"{metrics_radius_lin['quantization_error']:<15.6f}")

print(f"{'Topographic error':<30} {metrics_radius_exp['topographic_error']:<15.6f} "
      f"{metrics_radius_lin['topographic_error']:<15.6f}")

| Epoch | SOM1 (Radius Exponential Decay) | SOM1B (Radius Linear Decay + Cutoff) |
| ----- | :------------------: | :----------------------: |
| 20    | 5.02               | 4.92                   |
| 40    | 4.19               | 3.85                   |
| 60    | 3.51               | 2.77                   |
| 80    | 2.93               | 1.70                   |
| 100   | 2.45               | **1.0**                |
| 120   | 2.05               | **1.0**                |
| 140   | 1.71               | **1.0**                |
| 160   | 1.43               | **1.0**                |
| 180   | 1.20               | **1.0**                |
| 200   | **1.0**            | **1.0**                |

**Table Q3.1:** Comparison of the evolution of the neighborhood radius (σ(t)) between the original SOM and the SOM1B version.


| Metric | Radius Exponential Decay | Radius Linear Decay |
| ------ | :----------------: | :-----------: |
| QE | 0.002426 | 0.002013 |
| TE | 0.161905 | 0.142857 |

**Table Q3.2:** Comparison of the evolution of numeric criteria (Quantitization Erro **QE** and Topological Error **TE**) between the original SOM and the SOM1B version.

As shown in Table Q3.1, the linear decay introduced in SOM1B leads to a faster reduction of the neighborhood radius, reaching its minimum value around epoch 100, whereas the original SOM requires approximately 200 epochs. This behavior accelerates the transition from global ordering to local fine-tuning, promoting earlier local adaptation of the map.

According to the numerical results reported in Table Q3.2, the final quantization error obtained with the modified deviation curve is lower than that of the original SOM, indicating an improved representation of the input data. The topographic error also presents a slight reduction, suggesting a modest improvement in topological consistency.

From a training dynamics perspective, this modification can be regarded as beneficial, since the earlier reduction of the neighborhood radius limits the influence of distant neurons and reduces excessive smoothing in later epochs. As a consequence, the map converges more rapidly toward a locally consistent organization.

On the other hand, the faster decay of the neighborhood radius shortens the phase dedicated to global organization, which may negatively affect large-scale topological preservation in certain datasets. Therefore, the observed improvement is mainly associated with convergence speed and locality, rather than a guaranteed enhancement of global mapping quality.

Overall, under the adopted numerical criterion, the SOM1B version can be considered an improvement over the original SOM.

<br><br>
## **Q4:** Write the version SOM1C, where you change the change the normal distribution to other distribution of your
choice. Did you achieve improvements?

In the Self-Organizing Map algorithm, the neighborhood function defines how the update of the winning neuron propagates to its neighbors. In the original formulation, this influence is typically modeled using a Gaussian distribution, which provides a smooth and rapidly decaying spatial interaction.

In the SOM1C version, the Gaussian neighborhood function was replaced by an exponential distribution, while keeping the learning rate and neighborhood radius decay unchanged. This modification directly affects how the influence of the Best Matching Unit (BMU) spreads across the map during training.



The following code snippet illustrates the modification introduced in the SOM1C version, where the Gaussian neighborhood function was replaced by an exponential distribution.
```python
<our_som1C.py>
class SOM_C:
...
    @staticmethod
    def calculate_influence(distance, radius):
        #return np.exp(-distance / (2 * (radius ** 2)))
        # exponential distribution
        return np.exp(-np.sqrt(distance) / radius)

In [ ]:
### version SOM1C
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import patches as patches
import matplotlib.lines as mlines

# reading data
data = pd.read_csv("cash-crops-nepal.csv")
agri_data = data.iloc[np.random.permutation(len(data))]
trunc_data = agri_data[["Area", "Production", "Yield"]]
trunc_data = trunc_data / trunc_data.max()

from our_som1C import SOM_C
from our_som1 import SOM

# som = SOM(x_size, y_size, num_features)
np.random.seed(42)
agri_som_dist_exponential = SOM_C(6,6,3)

# Initial weights
init_fig = plt.figure()
agri_som_dist_exponential.show_plot(init_fig, 1, 0)
plt.show()

agri_som_dist_exponential.train(trunc_data.values,
          num_epochs=200,
          init_learning_rate=0.01
          )

#evaluate_som is defined in Questions 1/2 and reused here

metrics_dist_exponential = evaluate_som(agri_som_dist_exponential, trunc_data)

np.random.seed(42)
agri_som_dist_gaussian = SOM(6, 6, 3)
agri_som_dist_gaussian.train(trunc_data.values, num_epochs=200, init_learning_rate=0.01)
metrics_dist_gaussian = evaluate_som(agri_som_dist_gaussian, trunc_data)


print(f"\n{'Metric':<30} {'Dist Gaussian':<15} {'Dist Exponential':<15}")
print("-"*70)

print(f"{'QE':<30} {metrics_dist_gaussian['quantization_error']:<15.6f} "
      f"{metrics_dist_exponential['quantization_error']:<15.6f}")

print(f"{'Topographic error':<30} {metrics_dist_gaussian['topographic_error']:<15.6f} "
      f"{metrics_dist_exponential['topographic_error']:<15.6f}")


| Metric | Dist Gaussian | Dist Exponential |
| ------ | :----------------: | :-----------: |
| QE | 0.002417 | 0.002124 |
| TE | 0.152381 | 0.152381 |

**Table Q4.1:** Comparison of the evolution of numeric criteria (Quantitization Erro **QE** and Topological Error **TE**) between the original SOM and the SOM1C version.

Table Q4.1 reports the quantitative comparison between the Gaussian and exponential neighborhood functions using the numerical criteria defined in Question 1. The results show that the exponential neighborhood function yields a slightly lower quantization error, indicating a modest improvement in how the SOM prototypes represent the input data.

Regarding topological preservation, the topographic error remains unchanged for both neighborhood functions. This suggests that, under the considered training setup, replacing the Gaussian kernel with an exponential one does not significantly affect the global topological organization of the map.

From an algorithmic perspective, the exponential neighborhood function decays more slowly with distance from the BMU, allowing neurons farther away to retain influence during the update process, particularly in the early stages of training. While this can lead to smoother spatial adaptations, its impact on topological consistency appears to be limited for the present dataset.

Overall, the SOM1C modification can be considered a mild improvement in terms of quantization performance, while maintaining a similar level of topological consistency when compared to the original Gaussian-based SOM.


<br><br>
## **Q5$^*$:** Determine the mathematical conditions that ensure the convergence of equation (3) in page 14 of this slides.

We have the following equation
$$
w_k(t+1) = w_k(t)+\alpha(t)h_{ck}(t)[x(t)-w_k(t)]
$$
where $\alpha(t)$ is the learning-rate factor and $h_{ck}(t)$ is the neighborhood function, defined as:
$$
h_{ck}(t)= exp\left(\dfrac{-\|r_k-r_c\|^2}{σ(t)^2}\right)
$$

where $r_k$ and $r_c$ denote the coordinates of nodes $k$ and $c$, respectively on the two-dimensional lattice.

The first equation is a stochastic aproximation process and its convergence can be analyzed using classical results from stochastic approximation theory combined with properties specific to the SOM algorithm.

That way, the convergence of this process can be ensured under the following conditions:
1. **Learning-rate conditions (Robbins-Monro):**
$$
\sum_{t=1}^∞α(t)=∞,\space \sum_{t=1}^∞α^2(t)<∞
$$
These conditions guarantee sufficient exploration while preventing divergence.

2. **Neighborhood decay:**
$$
\sigma(t) → 0 \space \text{ as } \space t → ∞,
$$
which implies that
$$
h_{ck}(t) ⟶ δ_{ck},
$$
where $δ_{ck}$ corresponds to the Kronecker delta. This ensures that only the winning neuron is updated.

3. **Boundedness assumptions**
   - the input vectors $x(t)$ are drawn from a stationary distribution;
   - $\|x(t)\| \leq C < ∞$;
   - the weight vectors remain bounded: $\sup_{t} \|w_k(t)\| < \infty$.

Under these conditions, the first equation converges almost surely to a stable configuration corresponding to a local minimum of the distortion measure. As the neighborhood radius collapses, the algorithm asymptotically behaves like stochastic $k$-means clustering. While convergence of the weights is guaranteed, convergence to the global optimum is not.

<br><br>
## **Q6:** As explained in class, SOM can be seen as a Euler integration method for the corresponding ODE. Estimate the absolute error after N epochs.



The Self-Organizing Map (SOM) update rule can be interpreted as a numerical integration scheme for an underlying ordinary differential equation (ODE). Specifically, the standard SOM weight update,
$$
w_k(t+1) = w_k(t) + \alpha(t)\,h_{ck}(t)\,[x(t) - w_k(t)],
$$
corresponds to the explicit Euler method applied to the continuous-time ODE
$$
\frac{dw_k}{dt} = h_{ck}(t)\,[x(t) - w_k(t)] = f(t, w_k).
$$

In this formulation, the learning rate $\alpha(t)$ plays the role of the step size $h$ in the Euler discretization.

The Euler method is derived from the Taylor expansion, where the local truncation error corresponds to the neglected term $\frac{h^2}{2}y''(\xi)$. Therefore, the local truncation error at each step is:
$$
e_{\text{local}}(t) = \frac{\alpha(t)^2}{2} \left|\frac{d^2 w_k}{dt^2}\right|
$$

The second derivative can be computed as:
$$
\frac{d^2 w_k}{dt^2} = -h_{ck}(t)^2(x(t) - w_k(t))
$$

The absolute error after N epochs is estimated by accumulating the local errors:
$$
E_N = \sum_{t=1}^{T} e_{\text{local}}(t) = \sum_{t=1}^{T} \frac{\alpha(t)^2}{2} \left|h_{ck}(t)^2(x(t) - w_k(t))\right|
$$
where $T = N \cdot M$ is the total number of updates ($N$ epochs, $M$ samples per epoch).

In [ ]:
import numpy as np
from our_som1 import SOM

np.random.seed(42)
som = SOM(6, 6, 3)

total_error = 0.0
num_epochs = 200
init_learning_rate = 0.01
som.time_constant = num_epochs / np.log(som.init_radius)

for epoch in range(1, num_epochs + 1):
    radius = som.decay_radius(epoch)
    alpha = som.decay_learning_rate(init_learning_rate, epoch, num_epochs)

    for idx, row in trunc_data.iterrows():
        x_t = row.values
        bmu, bmu_idx = som.find_bmu(x_t)

        for i in range(6):
            for j in range(6):
                w_k = som.net[i, j, :]
                w_dist = np.sum((np.array([i, j]) - bmu_idx) ** 2)

                if w_dist <= radius ** 2:
                    h_ck = np.exp(-w_dist / (2 * radius ** 2))

                    # error local
                    local_error = (alpha**2 / 2) * np.sum(np.abs(h_ck**2 * (x_t - w_k)))
                    total_error += local_error

                    # weight update
                    som.net[i, j, :] = w_k + alpha * h_ck * (x_t - w_k)

print(f"Estimated absolute error (Euler): {total_error:.6f}")

<br><br>
## **Q7$^*$:**  How could you change the SOM method to use Runge-Kutta second order method? Is the improvements?

The classical Self-Organizing Map (SOM) algorithm can be understood through the lens of numerical analysis. Fundamentally, its weight update mechanism implements a numerical integration scheme for an underlying ordinary differential equation (ODE) that governs the evolution of the prototype vectors. In its standard formulation, this scheme corresponds to the **explicit Euler method**, which is a first-order numerical integrator.

The conventional SOM update rule,
$$
w_k(t+1) = w_k(t) + \alpha(t)\,h_{ck}(t)\,[x(t) - w_k(t)],
$$
can be interpreted as a direct time discretization of the continuous-time ODE
$$
\frac{dw_k}{dt} = h_{ck}(t)\,[x(t) - w_k(t)].
$$

Since the explicit Euler method is well known for its limited accuracy and conditional stability, a natural way to improve the numerical behavior of the algorithm is to adopt a higher-order integration scheme. In the context, replacing the Euler update with a **second-order Runge–Kutta method (RK2)** provides a meaningful enchancement. This two-stage approach, consisting of a  predictor step followed by a corrector step that averages the estimated slopes, yields a more faithful approximation of the underlying continuous dynamics.

When applied to the SOM setting, the RK2 update can be written as:
$$
\begin{aligned}
k_1 &= h_{ck}(t)\,[x(t) - w_k(t)], \\[4pt]
w_k^{*} &= w_k(t) + \alpha(t)\,k_1, \\[4pt]
k_2 &= h_{ck}(t)\,[x(t) - w_k^{*}], \\[4pt]
w_k(t+1) &= w_k(t) + \frac{\alpha(t)}{2}\,(k_1 + k_2).
\end{aligned}
$$

From a numerical analysis perspective, the main advantage of this modification lies in the order of the error. While the explicit Euler method exhibits a global truncation error of order $\mathcal{O}(h)$, the RK2 scheme achieves a global error of order $\mathcal{O}(h^2)$. As a result, for a given effective step size (or learning rate), the RK2-based SOM provides a more accurate integration of the governing ODE. In practice, this leads to smoother and more stable trajectories for the weight vectors during the learning process.

Although the RK2 variant introduces a modest increase in computational cost per iteration due to the additional function evaluation, this overhead is typically justified. The improved numerical stability and the potential gains in convergence robustness make the second-order Runge–Kutta approach a principled and effective refinement of the classical SOM algorithm.

The following code snippet illustrates the modification introduced in the version, where the conventional SOM update rule was replaced by the RK2 update.
```python
<our_som1_RK2.py>
class SOM_RK2:
...
    def train(self, data, num_epochs=100, init_learning_rate=0.01, resetWeights=False):
        ...
            for record in indices:
                ...
                for x in range(self.network_dimensions[0]):
                    for y in range(self.network_dimensions[1]):
                       ...
                        if w_dist <= radius ** 2:
                            # update weight vectors wk using Eq. (3)
                            influence = SOM_RK2.calculate_influence(w_dist, radius)
                            #second-order Runge–Kutta method (RK2)
                            new_w = SOM_RK2.rk2_update(weight, row_t, learning_rate, influence)
                            self.net[x, y, :] = new_w.reshape(1, self.num_features)

    @staticmethod
    def rk2_update(weight, x_t, alpha, influence):
        k1 = influence * (x_t - weight)
        w_star = weight + alpha * k1
        k2 = influence * (x_t - w_star)
        return weight + (alpha / 2.0) * (k1 + k2)

In [ ]:
### version SOM1_RK2
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import patches as patches
import matplotlib.lines as mlines

# reading data
data = pd.read_csv("cash-crops-nepal.csv")
agri_data = data.iloc[np.random.permutation(len(data))]
trunc_data = agri_data[["Area", "Production", "Yield"]]
trunc_data = trunc_data / trunc_data.max()

from our_som1_RK2 import SOM_RK2
from our_som1 import SOM

# som = SOM(x_size, y_size, num_features)
np.random.seed(42)
agri_som_RK2 = SOM(6,6,3)

# Initial weights
init_fig = plt.figure()
agri_som_RK2.show_plot(init_fig, 1, 0)
plt.show()

agri_som_RK2.train(trunc_data.values,
          num_epochs=200,
          init_learning_rate=0.01
          )

In summary, the classical SOM learning rule can be interpreted as an explicit Euler discretization of an underlying continuous-time dynamical system. By replacing this first-order integrator with a second-order Runge–Kutta scheme, the structure of the SOM algorithm remains unchanged, while the numerical integration of the governing ODE is improved.

The proposed RK2-based update introduces a more accurate approximation of the continuous dynamics, leading to smoother and more stable weight trajectories during training. Although this modification entails a slightly higher computational cost per iteration, the observed behavior suggests improved numerical stability and more robust convergence properties.

Therefore, from a numerical integration perspective, the use of a second-order Runge–Kutta method can be considered an improvement over the classical Euler-based SOM.

A quantitative assessment of the error reduction and convergence rate associated with this higher-order integration scheme is addressed in the following question.


<br><br>
## **Q8$^*$:** Estimate the absolute error after N epochs by using Q7.

Building upon Q7, where the SOM update rule was reformulated using the second-order Runge-Kutta (RK2) integration scheme, we now estimate the absolute error after $N$ epochs.

The RK2 method has a local truncation error of order $\mathcal{O}(h^3)$, derived from the Taylor expansion where the neglected term is $\frac{h^3}{6}y'''(\xi)$. For the SOM with RK2 update:
$$
e_{\text{local}}^{\text{RK2}}(t) = \frac{\alpha(t)^3}{6} \left|\frac{d^3 w_k}{dt^3}\right|
$$

The third derivative can be computed from the ODE $\frac{dw_k}{dt} = h_{ck}(t)(x(t) - w_k(t))$:
$$
\frac{d^3 w_k}{dt^3} = h_{ck}(t)^3(x(t) - w_k(t))
$$

The absolute error after N epochs using RK2 is:
$$
E_N^{\text{RK2}} = \sum_{t=1}^{T} e_{\text{local}}^{\text{RK2}}(t) = \sum_{t=1}^{T} \frac{\alpha(t)^3}{6} \left|h_{ck}(t)^3(x(t) - w_k(t))\right|
$$

In [ ]:
import pandas as pd
import numpy as np
from our_som1_RK2 import SOM_RK2

# data
data = pd.read_csv("cash-crops-nepal.csv")
trunc_data = data[["Area", "Production", "Yield"]]
trunc_data = trunc_data / trunc_data.max()

np.random.seed(42)
som_rk2 = SOM_RK2(6, 6, 3)
som_rk2.time_constant = 200 / np.log(som_rk2.init_radius)

total_error_rk2 = 0.0
num_epochs = 200
init_learning_rate = 0.01

for epoch in range(1, num_epochs + 1):
    radius = som_rk2.decay_radius(epoch)
    alpha = som_rk2.decay_learning_rate(init_learning_rate, epoch, num_epochs)

    for idx, row in trunc_data.iterrows():
        x_t = row.values
        bmu, bmu_idx = som_rk2.find_bmu(x_t)

        for i in range(6):
            for j in range(6):
                w_k = som_rk2.net[i, j, :]
                w_dist = np.sum((np.array([i, j]) - bmu_idx) ** 2)

                if w_dist <= radius ** 2:
                    h_ck = np.exp(-w_dist / (2 * radius ** 2))

                    # local error 3rd order rk
                    local_error = (alpha**3 / 6) * np.sum(np.abs(h_ck**3 * (x_t - w_k)))
                    total_error_rk2 += local_error

                    # weight update
                    k1 = h_ck * (x_t - w_k)
                    w_star = w_k + alpha * k1
                    k2 = h_ck * (x_t - w_star)
                    som_rk2.net[i, j, :] = w_k + (alpha / 2) * (k1 + k2)

print(f"Estimated absolute error after N=200 epochs (RK2): {total_error_rk2:.6f}")

<br><br>
## **Q9:** How would you combine the answers to Q1-Q8, in order to suggest an improved version?

<br><br>

## References

<a id="ref1">[1]</a>
https://hal.science/hal-02182882/document

<a id="ref2">[2]</a>
https://hal.science/hal-01796059/file/RIO_marie_mai_2018_HAL.pdf

<a id="ref3">[3]</a>
https://search.r-project.org/CRAN/refmans/aweSOM/html/somQuality.html

<a id="ref4">[4]</a>
https://www.columbia.edu/~ww2040/8100F16/RM51.pdf